## Importing Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import os
from pandasql import sqldf
import sqlite3
import math

# Change this in accordance with your datasource
os.chdir(r"C:\Users\Prashast\OneDrive\Desktop\Purdue\Course\MOD 3\Industry Practicum\Data")

## Reading Raw Data

In [3]:
Beginning_Inventory = pd.read_csv("Beginning inventory.csv")
Bagging_Activities = pd.read_csv("Bagging activities.csv")
Dispo_Activities = pd.read_csv("Dispo activities.csv")
Location_Transfers = pd.read_csv("Location transfers.csv")
Shipments = pd.read_csv("Shipments.csv")
Returns = pd.read_csv("Returns.csv")
Locations = pd.read_csv("Locations.csv")

In [4]:
# Change name of new columns based on the year you are forecasting shortfall for
xls = pd.ExcelFile("fy25Demand.xlsx")
df = pd.read_excel(xls, sheet_name=xls.sheet_names[0])
fy_25_demand = df.melt(id_vars=['ProdSub0', 'FY25 demand'], var_name='Warehouse', value_name='Proportion')
fy_25_demand['FY25 demand'] = fy_25_demand['FY25 demand'].apply(lambda x: np.round(x,0))
fy_25_demand['Absolute Demand'] = np.round((fy_25_demand['FY25 demand'] * fy_25_demand['Proportion']),0)
fy_25_demand.rename(columns={'ProdSub0': 'PRODSUB0', 
                             'FY25 demand': 'FY25_DEMAND', 
                             'Warehouse': 'LOCATION', 
                             'Proportion': 'PROPORTION', 
                             'Absolute Demand': 'ABSOLUTE_DEMAND'}, inplace=True)

## Data Transformation

In [5]:
Beginning_Inventory['ACTIVITYDATE'] = pd.to_datetime(Beginning_Inventory['ACTIVITYDATE'], format="%d-%m-%Y").dt.strftime("%Y-%m-%d")
Bagging_Activities['ACTIVITYDATE'] = pd.to_datetime(Bagging_Activities['ACTIVITYDATE'], format="%d-%m-%Y").dt.strftime("%Y-%m-%d")
Dispo_Activities['ACTIVITYDATE'] = pd.to_datetime(Dispo_Activities['ACTIVITYDATE'], format="%d-%m-%Y").dt.strftime("%Y-%m-%d")
Location_Transfers['ACTIVITYDATE'] = pd.to_datetime(Location_Transfers['ACTIVITYDATE'], format="%d-%m-%Y").dt.strftime("%Y-%m-%d")
Shipments['ACTIVITYDATE'] = pd.to_datetime(Shipments['ACTIVITYDATE'], format="%d-%m-%Y").dt.strftime("%Y-%m-%d")
Returns['ACTIVITYDATE'] = pd.to_datetime(Returns['ACTIVITYDATE'], format="%d-%m-%Y").dt.strftime("%Y-%m-%d")

In [6]:
###############################################################################################################################

## Safety Stock Level

#### Loading required files in the memory

In [7]:
# Create SQLite connection
conn = sqlite3.connect(":memory:")

# Register the POWER UDF on the connection
conn.create_function("POWER", 2, lambda x, y: math.pow(x, y))

Location_Transfers.to_sql("Location_Transfers", conn, index=False, if_exists="replace")
Beginning_Inventory.to_sql("Beginning_Inventory", conn, index=False, if_exists="replace")
Bagging_Activities.to_sql("Bagging_Activities", conn, index=False, if_exists="replace")
Shipments.to_sql("shipments", conn, index=False, if_exists="replace")

# Feed in latest years forecasted demand
fy_25_demand.to_sql("fy_25_demand", conn, index=False, if_exists="replace")

target_year = 2025

#### Calcualting Stock Shortfall Ratio for FISCAL YEAR, WAREHOUSE, PRODUCT

In [8]:
# Now run your SQL query using pd.read_sql_query:
query = f"""
with cte_beginning_inventory as
(
    SELECT 
         LOCATION,
         PRODSUB0,
         FISCALYEAR,
         SUM(NETUNITS) AS Beginning_Inventory_Units
    FROM
    Beginning_Inventory
    GROUP BY 1,2,3
)

,cte_bagging as
(
    SELECT 
         LOCATION,
         PRODSUB0,
         FISCALYEAR,
         SUM(NETUNITS) AS Bagged_Units
    FROM 
    Bagging_Activities
    GROUP BY 1,2,3
)

,cte_transfer_in AS 
(
    SELECT 
         TOLOCATION as LOCATION,
         PRODSUB0,
         FISCALYEAR,
         SUM(NETUNITS) AS Units_Transfered_In_Total,
         sum(case when cast(strftime('%m', ACTIVITYDATE) as integer) in (7,8,9,10,11,12) then NETUNITS end) as Units_Transfered_In_July_to_Dec
    FROM 
    Location_Transfers
    GROUP BY 1,2,3
)

,cte_transfer_out AS 
(
    SELECT 
         FROMLOCATION as LOCATION,
         PRODSUB0,
         FISCALYEAR,
         SUM(NETUNITS) AS Units_Transfered_Out_Total,
         sum(case when cast(strftime('%m', ACTIVITYDATE) as integer) in (7,8,9,10,11,12) then NETUNITS end) as Units_Transfered_Out_July_to_Dec
    FROM 
    Location_Transfers
    GROUP BY 1,2,3
)

,cte_ads as
(
	select
		 LOCATION
		,PRODSUB0
		,FISCALYEAR
		,Beginning_Inventory_Units
		,Bagged_Units
		,Units_Transfered_In_Total
		,Units_Transfered_In_July_to_Dec
		,Units_Transfered_Out_Total
		,Units_Transfered_Out_July_to_Dec
		,Units_Transfered_In_Sale_Season
		,Units_Transfered_Out_Sale_Season
		,Starting_Units_Sale_Season
		,case 
              when Starting_Units_Sale_Season = 0 then 0
              else min(round(Units_Transfered_In_Sale_Season / Starting_Units_Sale_Season,2),1) -- 85th percentile 
         end as stock_shortfall_ratio
		,case 
              when Starting_Units_Sale_Season = 0 then 0
              else min(round(Units_Transfered_Out_Sale_Season / Starting_Units_Sale_Season,2),1) --90th percentile
         end as stock_excess_ratio
	from
	(
		select
			 LOCATION
			,PRODSUB0
			,FISCALYEAR
			,Beginning_Inventory_Units
			,Bagged_Units
			,Units_Transfered_In_Total
			,Units_Transfered_In_July_to_Dec
            ,Units_Transfered_Out_Total
			,Units_Transfered_Out_July_to_Dec
			,case 
               when Units_Transfered_In_Total - Units_Transfered_In_July_to_Dec < 0 then 0
               else Units_Transfered_In_Total - Units_Transfered_In_July_to_Dec
             end as Units_Transfered_In_Sale_Season
			,case 
               when Units_Transfered_Out_Total - Units_Transfered_Out_July_to_Dec < 0 then 0
               else Units_Transfered_Out_Total - Units_Transfered_Out_July_to_Dec 
             end as Units_Transfered_Out_Sale_Season
			,case 
               when Beginning_Inventory_Units + Bagged_Units + Units_Transfered_In_July_to_Dec - Units_Transfered_Out_July_to_Dec < 0 then 0
               else Beginning_Inventory_Units + Bagged_Units + Units_Transfered_In_July_to_Dec - Units_Transfered_Out_July_to_Dec
             end as Starting_Units_Sale_Season
		from
		(
			select
				 coalesce(a.LOCATION,b.LOCATION,c.LOCATION,d.LOCATION) as LOCATION
				,coalesce(a.PRODSUB0,b.PRODSUB0,c.PRODSUB0,d.PRODSUB0) as PRODSUB0
				,coalesce(a.FISCALYEAR,b.FISCALYEAR,c.FISCALYEAR,d.FISCALYEAR) as FISCALYEAR
				,coalesce(a.Beginning_Inventory_Units,0) as Beginning_Inventory_Units
				,coalesce(b.Bagged_Units,0) as Bagged_Units
				,coalesce(c.Units_Transfered_In_Total,0) as Units_Transfered_In_Total
                ,coalesce(c.Units_Transfered_In_July_to_Dec,0) as Units_Transfered_In_July_to_Dec
				,coalesce(d.Units_Transfered_Out_Total,0) as Units_Transfered_Out_Total
                ,coalesce(d.Units_Transfered_Out_July_to_Dec,0) as Units_Transfered_Out_July_to_Dec
			from
			cte_beginning_inventory a
			full outer join
			cte_bagging b
			on
			a.LOCATION = b.LOCATION
			and
			a.PRODSUB0 = b.PRODSUB0
			and
			a.FISCALYEAR = b.FISCALYEAR
			full outer join
			cte_transfer_in c
			on
			coalesce(a.LOCATION,b.LOCATION) = c.LOCATION
			and
			coalesce(a.PRODSUB0,b.PRODSUB0) = c.PRODSUB0
			and
			coalesce(a.FISCALYEAR,b.FISCALYEAR) = c.FISCALYEAR
			full outer join
			cte_transfer_out d
			on
			coalesce(a.LOCATION,b.LOCATION,c.LOCATION) = d.LOCATION
			and
			coalesce(a.PRODSUB0,b.PRODSUB0,c.PRODSUB0) = d.PRODSUB0
			and
			coalesce(a.FISCALYEAR,b.FISCALYEAR,c.FISCALYEAR) = d.FISCALYEAR
		)
	)
)

select * from cte_ads;
"""
stock_shortfall = pd.read_sql_query(query, conn)
stock_shortfall.to_csv('stock_shortfall.csv', index = False)

#### Calculating safety stock unit for next year

In [9]:
stock_shortfall.to_sql("stock_shortfall", conn, index=False, if_exists="replace")

query = f"""
with cte_calc AS 
(
	select
		 LOCATION
		,PRODSUB0
		,FISCALYEAR
		,Beginning_Inventory_Units
		,Bagged_Units
		,Units_Transfered_In_Total
		,Units_Transfered_In_July_to_Dec
		,Units_Transfered_Out_Total
		,Units_Transfered_Out_July_to_Dec
		,Units_Transfered_In_Sale_Season
		,Units_Transfered_Out_Sale_Season
		,Starting_Units_Sale_Season
		,stock_shortfall_ratio
		,stock_excess_ratio
		,decay_weight
		,round(stock_shortfall_ratio * decay_weight,2) as shortfall_contribution
		,round(stock_excess_ratio * decay_weight,2) as excess_contribution
	from
	(
		SELECT 
			 LOCATION
			,PRODSUB0
			,FISCALYEAR
			,Beginning_Inventory_Units
			,Bagged_Units
			,Units_Transfered_In_Total
			,Units_Transfered_In_July_to_Dec
			,Units_Transfered_Out_Total
			,Units_Transfered_Out_July_to_Dec
			,Units_Transfered_In_Sale_Season
			,Units_Transfered_Out_Sale_Season
			,Starting_Units_Sale_Season
			,stock_shortfall_ratio
			,stock_excess_ratio
			,POWER(0.5, ({target_year - 1} - FISCALYEAR)) AS decay_weight
		FROM 
		stock_shortfall
        where
        FISCALYEAR < {target_year}
	)
)

,cte_next_year as
(
    select
		 LOCATION
		,PRODSUB0
		,case when denominator =0 then 0 else round(shortfall_numerator/denominator,2) end as predicted_shortfall_ratio
		,case when denominator =0 then 0 else round(excess_numerator/denominator,2) end as predicted_excess_ratio
    from
    (
		select
			 LOCATION
			,PRODSUB0
			,sum(shortfall_contribution) as shortfall_numerator
			,sum(excess_contribution) as excess_numerator
			,sum(decay_weight) as denominator
		from
		cte_calc
		group by 1,2
	)
)

,cte_final as
(
	select
		 Next_Year
		,LOCATION
		,PRODSUB0
		,ABSOLUTE_DEMAND
		,predicted_shortfall_ratio
		,predicted_excess_ratio
		,safety_stock_unit
		,excess_stock_unit
		,case
              when safety_stock_unit - excess_stock_unit < 0 then 0
              else safety_stock_unit - excess_stock_unit 
         end as net_safety_stock_unit
	from
	(
		select
			 Next_Year
			,LOCATION
			,PRODSUB0
			,ABSOLUTE_DEMAND
			,predicted_shortfall_ratio
			,predicted_excess_ratio
			,round(predicted_shortfall_ratio * ABSOLUTE_DEMAND * 1.35,0) as safety_stock_unit
			,round(predicted_excess_ratio * ABSOLUTE_DEMAND * 1.35,0) as excess_stock_unit
		from
		(
			select
				{target_year} as Next_Year
				,a.LOCATION
				,a.PRODSUB0
				,a.ABSOLUTE_DEMAND
				,coalesce(b.predicted_shortfall_ratio,0) as predicted_shortfall_ratio
				,coalesce(b.predicted_excess_ratio,0) as predicted_excess_ratio
			from
			fy_25_demand a
			left join
			cte_next_year b
			on
			a.LOCATION = b.LOCATION
			and
			a.PRODSUB0 = b.PRODSUB0
		)
	)
)

select  * from
cte_final
"""

safety_unit = pd.read_sql_query(query, conn)
safety_unit.to_csv('safety_unit.csv', index = False)

In [10]:
###############################################################################################################################

## Location Transfer Ratio & Late Transfer Ratio

In [12]:
query = f"""
WITH shipments_agg AS (
    SELECT
        LOCATION,
        FISCALYEAR,
        strftime('%m', ACTIVITYDATE) AS MonthNumber,
        CASE strftime('%m', ACTIVITYDATE)
            WHEN '01' THEN 'January'
            WHEN '02' THEN 'February'
            WHEN '03' THEN 'March'
            WHEN '04' THEN 'April'
            WHEN '05' THEN 'May'
            WHEN '06' THEN 'June'
            WHEN '07' THEN 'July'
            WHEN '08' THEN 'August'
            WHEN '09' THEN 'September'
            WHEN '10' THEN 'October'
            WHEN '11' THEN 'November'
            WHEN '12' THEN 'December'
        END AS Month,
        PRODSUB0,
        ABS(SUM(NETUNITS)) AS TotalUnitsShippedOut
    FROM shipments
    GROUP BY 
        LOCATION, 
        FISCALYEAR, 
        strftime('%m', ACTIVITYDATE),
        PRODSUB0
),
transfers_agg AS (
    SELECT
        TOLOCATION AS LOCATION,
        FISCALYEAR,
        strftime('%m', ACTIVITYDATE) AS MonthNumber,
        CASE strftime('%m', ACTIVITYDATE)
            WHEN '01' THEN 'January'
            WHEN '02' THEN 'February'
            WHEN '03' THEN 'March'
            WHEN '04' THEN 'April'
            WHEN '05' THEN 'May'
            WHEN '06' THEN 'June'
            WHEN '07' THEN 'July'
            WHEN '08' THEN 'August'
            WHEN '09' THEN 'September'
            WHEN '10' THEN 'October'
            WHEN '11' THEN 'November'
            WHEN '12' THEN 'December'
        END AS Month,
        PRODSUB0,
        SUM(NETUNITS) AS TotalUnitsTransferredIn
    FROM location_transfers
    GROUP BY 
        TOLOCATION, 
        FISCALYEAR, 
        strftime('%m', ACTIVITYDATE),
        PRODSUB0
)

SELECT
    COALESCE(s.LOCATION, t.LOCATION) AS Location,
    COALESCE(s.FISCALYEAR, t.FISCALYEAR) AS FiscalYear,
    COALESCE(s.Month, t.Month) AS Month,
    COALESCE(s.MonthNumber, t.MonthNumber) AS MonthNumber,
    COALESCE(s.PRODSUB0, t.PRODSUB0) AS ProductSub0,
    COALESCE(s.TotalUnitsShippedOut, 0) AS TotalUnitsShippedOut,
    COALESCE(t.TotalUnitsTransferredIn, 0) AS TotalUnitsTransferredIn
FROM shipments_agg s
LEFT JOIN transfers_agg t
    ON s.LOCATION = t.LOCATION
    AND s.FISCALYEAR = t.FISCALYEAR
    AND s.MonthNumber = t.MonthNumber
    AND s.PRODSUB0 = t.PRODSUB0

UNION ALL

SELECT
    COALESCE(s.LOCATION, t.LOCATION) AS Location,
    COALESCE(s.FISCALYEAR, t.FISCALYEAR) AS FiscalYear,
    COALESCE(s.Month, t.Month) AS Month,
    COALESCE(s.MonthNumber, t.MonthNumber) AS MonthNumber,
    COALESCE(s.PRODSUB0, t.PRODSUB0) AS ProductSub0,
    COALESCE(s.TotalUnitsShippedOut, 0) AS TotalUnitsShippedOut,
    COALESCE(t.TotalUnitsTransferredIn, 0) AS TotalUnitsTransferredIn
FROM transfers_agg t
LEFT JOIN shipments_agg s
    ON s.LOCATION = t.LOCATION
    AND s.FISCALYEAR = t.FISCALYEAR
    AND s.MonthNumber = t.MonthNumber
    AND s.PRODSUB0 = t.PRODSUB0

WHERE s.LOCATION IS NULL

ORDER BY FiscalYear, MonthNumber, ProductSub0;

"""

Transfer_Ratio = pd.read_sql_query(query, conn)
Transfer_Ratio.to_csv('Transfer_Ratio.csv', index = False)

In [ ]:
##############################################################################################################################